In [ ]:
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d, ConvexHull
import matplotlib.pyplot as plt
from matplotlib import cm
import multifidelityfunctions as mff
import multiLevelCoSurrogates as mlcs
from itertools import product
import pyDOE

In [ ]:
plot_dir = 'plots/SSCI2019a/'
mlcs.guaranteeFolderExists(plot_dir)

# Functions

In [ ]:
def sphere(X):
    return np.sum(1 - X**2, axis=1)

In [ ]:
def forrester(X):
    term1 = (6*X - 2)**2
    term2 = np.sin(12*X - 4)

    return np.sum(22 - (term1 * term2 + 6.03), axis=1)

In [ ]:
def ackley(X):
    tau = 2*np.pi
    
    sum_squares = np.sum(X**2, axis=1)
    sum_cos = np.sum(np.cos(tau*X), axis=1)
    
    term1 = -20*np.exp(-0.2 * np.sqrt(0.5*sum_squares))
    term2 = -np.exp(0.5*sum_cos)
    return term1 + term2 + np.e + 20

In [ ]:
# @mff.row_vectorize
# def himmelblau(xx):
#     pass

himmelblau = mff.himmelblau_hf

In [ ]:
def rastrigin(X):
    N = X.shape[1]
    A = 10
    tau = 2*np.pi
    
    term1 = A * N
    term2a = X**2
    term2b = A * np.cos(tau*X)
    term2 = np.sum(term2a - term2b, axis=1)
    return term1 + term2

In [ ]:
def sombrero(X):
#     a = 4
#     y = ((1 - (a*X)**2) * np.exp(-(a*X)**2 / 2) + .5) / 1.5
    term1 = np.sqrt(np.sum(X**2, axis=1))
    return (np.sin(term1) / term1) + .25

# Bounded Voronoi Example

In [ ]:
def voronoi_volumes(v):
    return np.array([
        ConvexHull(v.vertices[indices]).volume
        for indices in v.filtered_regions
    ])

In [ ]:
def in_box(x, bounding_box):
    return np.logical_and(np.logical_and(bounding_box[0] <= x[:, 0],
                                         x[:, 0] <= bounding_box[1]),
                          np.logical_and(bounding_box[2] <= x[:, 1],
                                         x[:, 1] <= bounding_box[3]))


def voronoi(x, bounding_box):
    # Select towers inside the bounding box
    i = in_box(x, bounding_box)
    # Mirror points
    points_center = x[i, :]
    points_left = np.copy(points_center)
    points_left[:, 0] = bounding_box[0] - (points_left[:, 0] - bounding_box[0])
    points_right = np.copy(points_center)
    points_right[:, 0] = bounding_box[1] + (bounding_box[1] - points_right[:, 0])
    points_down = np.copy(points_center)
    points_down[:, 1] = bounding_box[2] - (points_down[:, 1] - bounding_box[2])
    points_up = np.copy(points_center)
    points_up[:, 1] = bounding_box[3] + (bounding_box[3] - points_up[:, 1])
    points = np.concatenate((points_center,
                             points_left,
                             points_right,
                             points_down,
                             points_up), axis=0)
    # Compute Voronoi
    vor = Voronoi(points)
    # Filter by points_center
    vor.filtered_points = points_center
    vor.filtered_regions = np.array(vor.regions)[vor.point_region[:vor.npoints//5]]

    return vor


n_points = 10
np.random.seed(20200501)
x = np.random.rand(n_points, 2)
bounding_box = np.array([0., 1., 0., 1.]) # [x_min, x_max, y_min, y_max]


vor = voronoi(x, bounding_box)
volumes = voronoi_volumes(vor)
print(sum(volumes))

fig = plt.figure(figsize=(10,6))
ax = fig.gca()
# Plot initial points
# for i, p in enumerate(vor.filtered_points):
#     ax.text(*p, i)
    
ax.plot(vor.filtered_points[:, 0], vor.filtered_points[:, 1], 'b.')
# Plot ridges points
for i, (region, point, vol) in enumerate(zip(vor.filtered_regions, vor.filtered_points, volumes)):
#     vertices = vor.vertices[region, :]
#     ax.plot(vertices[:, 0], vertices[:, 1], 'go')
    
    vertices = vor.vertices[region + [region[0]], :]
    ax.plot(vertices[:, 0], vertices[:, 1], 'k-')

    polygon = vor.vertices[region]
    ax.fill(*zip(*polygon), alpha=0.4, label=i)
    
#     ax.text(*point, str(round(vol, 3)))


ax.set_xlim([-0.1, 1.1])
ax.set_ylim([-0.1, 1.1])
# plt.legend()
plt.tight_layout()
plt.savefig(f"{plot_dir}bounded_voronoi.png")
plt.savefig(f"{plot_dir}bounded_voronoi.pdf")
plt.show()

# 2D FSS-weighted error example

In [ ]:
test_func = sombrero
lbound = -10
ubound = 10


In [ ]:
step = 0.1
surface_params = {'l_bound': [lbound, lbound], 'u_bound': [ubound, ubound], 'step': [step, step]}
surf = mlcs.createsurface(test_func, **surface_params)

mlcs.plotsurfaces([surf])

In [ ]:
x = np.array(list(product([0.0, 0.1, 0.55, 0.9], repeat=2))).reshape(-1,2)
x = mlcs.rescale(x, range_in=(0,1), range_out=(lbound, ubound))

archive = mlcs.CandidateArchive(ndim=2)#, fidelities=['high', 'low', 'high-low'])
archive.addcandidates(x, test_func(x))#, fidelity='high')

surr = mlcs.Surrogate.fromname('Kriging', archive, kernel='Matern')
surr.retrain()

# y_surr = surr.predict(xrange.reshape(-1,1))
surr_surf = mlcs.createsurface(surr.predict, **surface_params)
abs_err = np.abs((surf-surr_surf).Z)
scale = (surf.Z/np.nanmax(surf.Z))*.98 + .01


abserr_surf = mlcs.Surface(surf.X, surf.Y, abs_err)
scaled_surf = mlcs.Surface(surf.X, surf.Y, abs_err*scale)

mlcs.plotsurfaces([surf, surr_surf, abserr_surf, scaled_surf],
                  titles=['True function', 'GP model', 'Abs. error', 'f(x)-weighted abs. error'],
                  shape=(2,2), **surface_params, save_as=f'{plot_dir}2d_forrester_wireframe.pdf')

In [ ]:
sample_size = 2000

bounding_box = np.array([lbound, ubound, lbound, ubound]) # [x_min, x_max, y_min, y_max]
# plt.figure(figsize=(6,12))
# plt.figure(figsize=(18,4))

###########

np.random.seed(20160501)
test_sample = np.random.rand(sample_size,2)
test_sample = mlcs.rescale(test_sample, range_in=(0,1), range_out=(lbound, ubound))

vor = voronoi(test_sample, bounding_box)
volumes = voronoi_volumes(vor)
sq_err = (test_func(test_sample) - surr.predict(test_sample))**2
weighted_errors = sq_err / volumes


fig = plt.figure(figsize=(8, 8))
ax = fig.gca()
# Plot initial points
# ax.plot(vor.filtered_points[:, 0], vor.filtered_points[:, 1], 'b.')

for i, (region, point, err) in enumerate(zip(vor.filtered_regions, vor.filtered_points, weighted_errors)):
    # Plot ridges points
#     vertices = vor.vertices[region, :]
#     ax.plot(vertices[:, 0], vertices[:, 1], 'go')
    # Plot ridges
    vertices = vor.vertices[region + [region[0]], :]
    ax.plot(vertices[:, 0], vertices[:, 1], 'k-')
    # Fill regions
    polygon = vor.vertices[region]
    ax.fill(*zip(*polygon), color=cm.viridis_r(err/np.max(weighted_errors)))


ax.set_xlim([lbound, ubound])
ax.set_ylim([lbound, ubound])
plt.tight_layout()
# plt.savefig(f"{plot_dir}error_scaled_voronoi_n={sample_size}.png")
plt.show()

In [ ]:
def plot_voronoi_onaxis(ax, vor, cvals, lbound, ubound, title, plot_ridges=False):
    for i, (region, point, cval) in enumerate(zip(vor.filtered_regions, vor.filtered_points, cvals)):
        polygon = vor.vertices[region]
        ax.fill(*zip(*polygon), color=cm.viridis_r(cval))

    if plot_ridges:
        for region in vor.filtered_regions:
            # Plot ridges
            vertices = vor.vertices[region + [region[0]], :]
            ax.plot(vertices[:, 0], vertices[:, 1], 'k-')

    ax.set_title(title)
    ax.set_xlim([lbound, ubound])
    ax.set_ylim([lbound, ubound])

In [ ]:
def sampling_area_distribution(sample_size, show=False):

    ###########

    np.random.seed(20160501)
    test_sample = np.random.rand(sample_size,2)
    test_sample = mlcs.rescale(test_sample, range_in=(0,1), range_out=(lbound, ubound))

    vor1 = voronoi(test_sample, bounding_box)
    volumes1 = voronoi_volumes(vor1)
    sq_err = (test_func(test_sample) - surr.predict(test_sample))**2
    # weighted_errors1 = sq_err * volumes1

    ###############

    np.random.seed(20160501)
    test_sample = pyDOE.lhs(2, sample_size)
    test_sample = mlcs.rescale(test_sample, range_in=(0,1), range_out=(lbound, ubound))

    vor2 = voronoi(test_sample, bounding_box)
    volumes2 = voronoi_volumes(vor2)
    sq_err = (test_func(test_sample) - surr.predict(test_sample))**2
    # weighted_errors2 = sq_err * volumes2

    ##################

    np.random.seed(20160501)
    test_sample = mlcs.sample_by_function(test_func, ndim=2, n_samples=sample_size, 
                                          minimize=False, range_in=(lbound,ubound), range_out=(-0.2,1))

    vor3 = voronoi(test_sample, bounding_box)
    volumes3 = voronoi_volumes(vor3)
    sq_err = (test_func(test_sample) - surr.predict(test_sample))**2
    # weighted_errors3 = sq_err * volumes3

    ###################

    max_volume = np.max([
        np.max(volumes1),
        np.max(volumes2),
        np.max(volumes3),
    ])


    fig = plt.figure(figsize=(18, 6))
    ridges = False

    ax = plt.subplot(131)
    plot_voronoi_onaxis(ax, vor1, volumes1/max_volume, lbound, ubound, 'Uniform Random', plot_ridges=ridges)

    ax = plt.subplot(132)
    plot_voronoi_onaxis(ax, vor2, volumes2/max_volume, lbound, ubound, 'LHS', plot_ridges=ridges)

    ax = plt.subplot(133)
    plot_voronoi_onaxis(ax, vor3, volumes3/max_volume, lbound, ubound, 'FSS', plot_ridges=ridges)


    plt.tight_layout()
    plt.savefig(f"{plot_dir}tri_volume_voronoi_n={sample_size}.png")
    plt.savefig(f"{plot_dir}tri_volume_voronoi_n={sample_size}.pdf")
    if show:
        plt.show()

In [ ]:
def weighted_error_visualization(sample_size, show=False):

    ###########

    np.random.seed(20160501)
    test_sample = np.random.rand(sample_size,2)
    test_sample = mlcs.rescale(test_sample, range_in=(0,1), range_out=(lbound, ubound))

    vor1 = voronoi(test_sample, bounding_box)
    volumes1 = voronoi_volumes(vor1)
    sq_err1 = (test_func(test_sample) - surr.predict(test_sample))**2
    print(np.mean(sq_err1))
    weighted_errors1 = sq_err1 / volumes1

    ###############

    np.random.seed(20160501)
    test_sample = pyDOE.lhs(2, sample_size)
    test_sample = mlcs.rescale(test_sample, range_in=(0,1), range_out=(lbound, ubound))

    vor2 = voronoi(test_sample, bounding_box)
    volumes2 = voronoi_volumes(vor2)
    sq_err2 = (test_func(test_sample) - surr.predict(test_sample))**2
    print(np.mean(sq_err2))
    weighted_errors2 = sq_err2 / volumes2

    ##################

    np.random.seed(20160501)
    test_sample = mlcs.sample_by_function(test_func, ndim=2, n_samples=sample_size, 
                                          minimize=False, range_in=(lbound, ubound), range_out=(-0.2,1))

    vor3 = voronoi(test_sample, bounding_box)
    volumes3 = voronoi_volumes(vor3)
    sq_err3 = (test_func(test_sample) - surr.predict(test_sample))**2
    print(np.mean(sq_err3))
    weighted_errors3 = sq_err3 / volumes3

    ###################

    max_weight = np.max([
        np.max(weighted_errors1),
        np.max(weighted_errors2),
        np.max(weighted_errors3),
    ])


    fig = plt.figure(figsize=(18, 6))
    ridges = False

    ax = plt.subplot(131)
    plot_voronoi_onaxis(ax, vor1, weighted_errors1/max_weight, lbound, ubound, 'Uniform Random', plot_ridges=ridges)

    ax = plt.subplot(132)
    plot_voronoi_onaxis(ax, vor2, weighted_errors2/max_weight, lbound, ubound, 'LHS', plot_ridges=ridges)

    ax = plt.subplot(133)
    plot_voronoi_onaxis(ax, vor3, weighted_errors3/max_weight, lbound, ubound, 'FSS', plot_ridges=ridges)


    plt.tight_layout()
    plt.savefig(f"{plot_dir}tri_error_scaled_voronoi_n={sample_size}.png")
    plt.savefig(f"{plot_dir}tri_error_scaled_voronoi_n={sample_size}.pdf")
    if show:
        plt.show()

In [ ]:
sizes =  [  500,  1000, 2500,  5000, 10000]
toshow = [False, False, True, False, False]

for sample_size, show in zip(sizes, toshow):
    sampling_area_distribution(sample_size)
    weighted_error_visualization(sample_size)